In [ ]:
import yfinance as yf
import pandas as pd
import polars as pl

pl.Config.set_tbl_cols(20)
pl.Config.set_tbl_rows(20)

## Baixando histórico

Dica: `period` (ex: `1d`, `5d`, `1mo`, `6mo`, `1y`) ou `start/end`.

In [ ]:
ticker = "B3SA3.SA"
period = "1mo"

stock = yf.Ticker(ticker)
history_pd = stock.history(period=period)
history_pd.head()

## Convertendo para Polars e limpando colunas

O retorno do `history()` vem como pandas com o índice sendo a data. Aqui nós damos `reset_index()` para trazer a data para coluna e renomeamos campos.

In [ ]:
history_pl = pl.from_pandas(history_pd.reset_index())

history_pl = history_pl.drop([c for c in ["Dividends", "Stock Splits"] if c in history_pl.columns])
history_pl = history_pl.rename({
    "Date": "date",
    "Open": "open",
    "High": "high",
    "Low": "low",
    "Close": "close",
    "Volume": "volume",
})

history_pl

## Exemplos rápidos de análise

In [ ]:
(
    history_pl
    .with_columns(
        pl.col("close").pct_change().alias("close_return")
    )
    .select(["date", "open", "high", "low", "close", "volume", "close_return"])
    .tail(10)
)

## Exportando (opcional)

Útil para criar um dataset local (ex.: para camada raw) usando Parquet.

In [ ]:
out_path = f"../data/{ticker}_{period}.parquet"
# history_pl.write_parquet(out_path)
out_path